<a href="https://colab.research.google.com/github/pgordin/OptDisc2025/blob/main/Lab1_LP_ILP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linki do tutoriali

[Programowanie liniowe](https://medium.com/@chongjingting/4-ways-to-solve-linear-programming-in-python-b4af36b7894d)

[Programowanie całkowitoliczbowe w PULP](https://medium.com/@gazalashaikh999/mixed-integer-programming-cfe0c196e875)

[Automatyzacja programowania całkowitoliczbowego w Google-OR
](https://developers.google.com/optimization/mip/mip_var_array?hl=pl)

# Zadanie 1.
Na przykładzie ciągłego problemu plecakowego sprawdzić (i przetestować) SciPy, PuLP i Google OR-Tools.

**1. ScipPy**

In [2]:
import numpy as np
from scipy.optimize import linprog

Dane dla przedmiotów

In [3]:
#wartości przedmiotów
values = [60, 100, 120]

#wagi przedmiotów
weights = [10, 20, 30]

#maksymalna waga plecaka
W = 50

#liczba przedmiotów
n = len(values)

Funkcja celu

In [4]:
c = [-v for v in values]

Wprowadzone ograniczenia

In [5]:
#ograniczenie sumy wag
A = [weights]

#ograniczenie na maksymalną wagę plecaka
b = [W]

# ograniczenie, że x_i jest w przedziale [0, 1]
bounds = [(0, 1) for _ in range(n)]

Rozwiązanie problemu

In [6]:
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
if result.success:
    print("Optymalne rozwiązanie")
    print(f"Wybrane przedmioty: {np.round(result.x, 2)}")
    print(f"Łączna wartość plecaka: {-result.fun}")
else:
    print("Nie znaleziono rozwiązania")

Optymalne rozwiązanie
Wybrane przedmioty: [1.   1.   0.67]
Łączna wartość plecaka: 240.0


**2. PuLP**

In [8]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 79.5 MB/s eta 0:00:00


In [9]:
from pulp import *

In [10]:
model = pulp.LpProblem('linear_programming', LpMaximize)

get solver

In [11]:
solver = getSolver('PULP_CBC_CMD')

Zmienne decyzyjne

In [12]:
# ilosci wzietych poszczegolnych produktów
x1 = LpVariable('x1', lowBound = 0, cat = 'Continuous')
x2 = LpVariable('x2', lowBound = 0, cat = 'Continuous')
x3 = LpVariable('x3', lowBound = 0, cat = 'Continuous')
x4 = LpVariable('x4', lowBound = 0, cat = 'Continuous')
x5 = LpVariable('x5', lowBound = 0, cat = 'Continuous')

#pojemnosc plecaka
pojemnosc = 1200

#maksymalne możliwe ilości do wzięcia
quant = [10,7,8,5,6]

#masy jednostkowe produktów
mass = [46, 40, 42, 38, 10]

#ceny jednostkowe produktów
values = [12, 19, 19, 15, 8]

Cel

In [13]:
model += values[0]*x1+values[1]*x2+values[2]*x3+values[3]*x4+values[4]*x5

Wprowadzenie ograniczeń

In [14]:
model += mass[0]*x1+mass[1]*x2+mass[2]*x3+mass[3]*x4+mass[4]*x5 <= pojemnosc

for i in range(1,6):
    model += eval('x{}'.format(i)) <= quant[i-1]

Rozwiązanie

In [15]:
results = model.solve(solver=solver)

# print results
if LpStatus[results] == 'Optimal': print('The solution is optimal.')
print(f'Objective value: z* = {value(model.objective)}')
print(f'Mass value: z* = {mass[0]*value(x1)+mass[1]*value(x2)+mass[2]*value(x3)+mass[3]*value(x4)+mass[4]*value(x5)}')
print(f'Solution: x1* = {value(x1)}, x2* = {value(x2)}, x3* = {value(x3)}, x4* = {value(x4)}, x5* = {value(x5)}')

The solution is optimal.
Objective value: z* = 495.1304352
Mass value: z* = 1200.0000016
Solution: x1* = 7.2608696, x2* = 7.0, x3* = 8.0, x4* = 5.0, x5* = 6.0


**3. Google OR-Tools**

- - -

# Zadanie 2.
Jak wygląda programowanie całkowitoliczbowe w tych pakietach? Rozwiązać (zwykły) problem plecakowy.


- - -

**1. ScipPy**

SciPy nie obsługuje programowania całkowitoliczbowego bezpośrednio, więc do rozwiązania tego typu problemu możemy sprawdzić wszystkie kombinacje wybranych przedmiotów i znaleźć najlepszą możliwą całkowitoliczbową wartość.

In [16]:
import numpy as np
from scipy.optimize import linprog
from itertools import combinations

Dane dla przedmiotów

In [17]:
#wartości przedmiotów
values = [60, 100, 120]

#wagi przedmiotów
weights = [10, 20, 30]

#maksymalna waga plecaka
W = 50

#liczba przedmiotów
n = len(values)

Funkcja celu

In [18]:
c = [-v for v in values]

Ograniczenia dla funkcji

In [19]:
#ograniczenie wagowe
A = [weights]

#ograniczenie na maksymalną wagę plecaka
b = [W]

#ograniczenia x_i ∈ {0, 1}
bounds = [(0, 1) for _ in range(n)]

Rozwiązanie problemu

In [20]:
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
if result.success:
    #zaokrąglamy i sprawdzamy wszystkie kombinacje
    best_value = 0
    best_combination = None

    #przegląd wszystkich kombinacji przedmiotów
    for r in range(1, n + 1):
        for subset in combinations(range(n), r):
            subset_weight = sum(weights[i] for i in subset)
            subset_value = sum(values[i] for i in subset)

            if subset_weight <= W and subset_value > best_value:
                best_value = subset_value
                best_combination = subset

    selected_items = [1 if i in best_combination else 0 for i in range(n)]

    print("Optymalne rozwiązanie")
    print(f"Wybrane przedmioty: {selected_items}")
    print(f"Łączna wartość plecaka: {best_value}")
else:
    print("Nie znaleziono rozwiązania")

Optymalne rozwiązanie
Wybrane przedmioty: [0, 1, 1]
Łączna wartość plecaka: 220


**2. PuLP**

In [21]:
!pip install pulp

In [22]:
from pulp import *

model = pulp.LpProblem('linear_programming', LpMaximize)

# get solver
solver = getSolver('PULP_CBC_CMD')

# declare decision variables - ilosci wzietych poszczegolnych produktów
x1 = LpVariable('x1', lowBound = 0, cat = 'Integer')
x2 = LpVariable('x2', lowBound = 0, cat = 'Integer')
x3 = LpVariable('x3', lowBound = 0, cat = 'Integer')
x4 = LpVariable('x4', lowBound = 0, cat = 'Integer')
x5 = LpVariable('x5', lowBound = 0, cat = 'Integer')

#pojemnosc plecaka
pojemnosc = 1200

#maksymalne możliwe ilości do wzięcia
quant = [10,7,8,5,6]

#masy jednostkowe produktów
mass = [46, 40, 42, 38, 10]

#ceny jednostkowe produktów
values = [12, 19, 19, 15, 8]

# declare objective
model += values[0]*x1+values[1]*x2+values[2]*x3+values[3]*x4+values[4]*x5

# declare constraints
model += mass[0]*x1+mass[1]*x2+mass[2]*x3+mass[3]*x4+mass[4]*x5 <= pojemnosc

for i in range(1,6):
    model += eval('x{}'.format(i)) <= quant[i-1]

# solve
results = model.solve(solver=solver)

# print results
if LpStatus[results] == 'Optimal': print('The solution is optimal.')
print(f'Objective value: z* = {value(model.objective)}')
print(f'Mass value: z* = {mass[0]*value(x1)+mass[1]*value(x2)+mass[2]*value(x3)+mass[3]*value(x4)+mass[4]*value(x5)}')
print(f'Solution: x1* = {value(x1)}, x2* = {value(x2)}, x3* = {value(x3)}, x4* = {value(x4)}, x5* = {value(x5)}')

The solution is optimal.
Objective value: z* = 492.0
Mass value: z* = 1188.0
Solution: x1* = 7.0, x2* = 7.0, x3* = 8.0, x4* = 5.0, x5* = 6.0


**3. Google OR-Tools**